In [1]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from subprocess import check_output
import utils
from utils import keras_baselilne
from utils import resnet
%matplotlib inline

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [14]:
#Load data
train = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/train.json")
valid = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/valid.json")
test = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train/test.json")
train_final = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/train_final.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
valid.inc_angle = valid.inc_angle.replace('na', 0)
valid.inc_angle = valid.inc_angle.astype(float).fillna(0.0)
train_final.inc_angle = train_final.inc_angle.replace('na', 0)
train_final.inc_angle = train_final.inc_angle.astype(float).fillna(0.0)
#train_final = train_final[~(train_final.inc_angle=='na')]
print("done!")

done!


In [9]:

# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band2)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Valid data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_2"]])
X_valid = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band2)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_valid = np.array(valid.inc_angle)
y_valid = np.array(valid["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band2)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)


# Train_final data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_final["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_final["band_2"]])
X_train_final = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band2)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train_final = np.array(train_final.inc_angle)
y_train_final = np.array(train_final["is_iceberg"])

In [10]:
for i in range(len(X_angle_train_final)):
    if type(X_angle_train_final[i]) != np.float64 and type(X_angle_train_final[i]) != np.float:
        X_angle_train_final[i] = round(np.random.normal(39.2037,3.813),3)
#np.random.normal(39.2037,3.813,size=61))

In [11]:
[obj for obj in X_angle_train_final if type(obj) != np.float64 and type(obj) != float]

[]

In [12]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
    

model = utils.keras_baselilne_batch()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_1 (InputLayer)                (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 75, 75, 3)    12          X_1[0][0]                        
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 73, 73, 16)   448         batch_normalization_15[0][0]     
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 73, 73, 16)   64          conv2d_9[0][0]                   
__________________________________________________________________________________________________
conv2d_10 

In [13]:
file_path = "/mnt/extDisk/courses/data/statoil/data/models/.noise_batch.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

model.load_weights(filepath=file_path)
print("Train evaluate:")
print(model.evaluate([X_train,X_angle_train], y_train, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate([X_train_final,X_angle_train_final], y_train_final, verbose=1, batch_size=200))

Train evaluate:


ValueError: could not convert string to float: na

In [42]:
prediction_train = model.predict([X_train_final,X_angle_train_final], verbose=1, batch_size=200)

802/802 [==============================] - 0s 174us/step


In [43]:
submission = pd.DataFrame({'id': train_final["id"], 'is_iceberg': prediction_train.reshape((prediction_train.shape[0]))})
submission.head(10)

,id,is_iceberg
100,66348d03,0.000797
1000,7052a617,0.407515
1001,3062fca8,0.998114
1002,4ea48c18,0.000884
101,f9209504,0.000806
1012,204afe46,0.697797
1014,47f0178a,0.943507
1015,47377a6d,0.877789
1016,d782d1a6,0.012856
1018,c04a768c,0.949111


In [44]:
submission.to_csv('/mnt/extDisk/courses/data/statoil/data/valid_results/flip_angle.csv',index=False)

In [52]:
submission_2 = pd.read_csv('/mnt/extDisk/courses/data/statoil/data/valid_results/noise_mix_learning_angle.csv')
submission_2.head()

,id,is_iceberg
0,66348d03,0.000713
1,7052a617,0.074851
2,3062fca8,0.999131
3,4ea48c18,0.000146
4,f9209504,0.002119


In [53]:
submission['is_iceberg'] = np.mean([submission['is_iceberg'].values,submission_2['is_iceberg'].values],axis=0)

In [54]:
log_loss(y_train_final,submission['is_iceberg'])

0.22286875489220684

In [11]:
submissions_dfs = []
paths = ['flip_angle.csv', 'ensamble_0211_angle.csv','pseudo_angle.csv', 'black_box_angle.csv','noise_box_angle.csv']
#paths = ['data_aug/noise_mix_clean_learning_rate_1.csv','ensamble/stacking_0612.csv']
for path in paths:
    submission_df = pd.read_csv("/mnt/extDisk/courses/data/statoil/data/valid_results/" + path)
    submission_df = submission_df.sort('id')
    submissions_dfs.append(submission_df)

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  


In [12]:
for i in range(len(submissions_dfs)):
    submissions_dfs[i] = submissions_dfs[i].rename(columns={'is_iceberg':'is_iceberg_' +str(i)})

concat_sub = submissions_dfs[0]

for i in range(1,len(submissions_dfs)):
    concat_sub = concat_sub.merge(submissions_dfs[i],on='id')

#concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5
0,004300e6,0.582211,0.758660,0.587314,0.886724,0.937983,0.847447
1,00e759b3,0.000149,0.000404,0.001110,0.000030,0.000625,0.000013
2,00f3862a,0.804578,0.484320,0.252155,0.963272,0.177252,0.779778
3,00f7ef1c,0.995288,0.989437,0.999920,0.999774,0.999383,0.999886
4,010355ab,0.583049,0.545799,0.506296,0.487851,0.459540,0.350795


In [13]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:].median(axis=1)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median
0,004300e6,0.582211,0.758660,0.587314,0.886724,0.937983,0.847447,0.937983,0.582211,0.765067,0.765067
1,00e759b3,0.000149,0.000404,0.001110,0.000030,0.000625,0.000013,0.001110,0.000013,0.000432,0.000404
2,00f3862a,0.804578,0.484320,0.252155,0.963272,0.177252,0.779778,0.963272,0.177252,0.575235,0.575235
3,00f7ef1c,0.995288,0.989437,0.999920,0.999774,0.999383,0.999886,0.999920,0.989437,0.996631,0.999383
4,010355ab,0.583049,0.545799,0.506296,0.487851,0.459540,0.350795,0.583049,0.350795,0.483397,0.487851


In [14]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [15]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub['is_iceberg'].head()

0    0.765067
1    0.000013
2    0.575235
3    0.999920
4    0.487851
Name: is_iceberg, dtype: float64

In [23]:
concat_sub.to_csv('/mnt/extDisk/courses/data/statoil/data/valid_results/stacking_angle.csv',index=False)

In [50]:
train_final.sort('id').head()

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


,band_1,band_2,id,inc_angle,is_iceberg
1309,"[-26.907173, -27.247885, -25.379496, -25.37954...","[-29.186039, -27.972126, -28.358276, -28.76238...",004300e6,42.5591,1
730,"[-15.341173, -14.21162, -15.837792, -15.923442...","[-22.030249, -20.892223, -21.046036, -24.18857...",00e759b3,0.0000,0
1147,"[-20.174095, -20.66132, -21.917173, -21.917212...","[-27.022549, -25.438961, -21.002024, -21.72630...",00f3862a,45.2814,1
896,"[-29.086002, -29.490072, -24.570494, -24.11330...","[-23.67878, -24.338787, -29.490143, -29.490211...",00f7ef1c,34.4748,1
829,"[-25.889074, -25.889074, -25.604317, -25.60437...","[-28.583046, -28.583046, -31.909725, -28.19704...",010355ab,39.9784,1


In [18]:
log_loss(train_final.sort('id')['is_iceberg'],np.mean([concat_sub['is_iceberg'],submission_2.sort('id')['is_iceberg']],axis=0))

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


0.20497918107697513

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


1309    1
730     0
1147    1
896     1
829     1
107     0
848     0
636     0
1509    0
1199    0
246     1
1067    0
1565    0
1166    0
1179    1
1057    0
1060    1
1575    0
1162    0
508     0
1318    0
1405    0
1368    0
1382    0
1350    0
640     0
80      0
1453    0
1492    1
945     0
       ..
1134    0
1297    1
740     0
1117    0
145     0
1538    0
761     1
728     1
1176    0
101     0
362     0
890     1
499     0
28      1
133     1
1076    1
887     0
643     1
756     1
1275    0
613     1
684     0
205     0
525     1
734     1
1363    1
1047    0
1251    0
787     1
886     1
Name: is_iceberg, dtype: int64